In [ ]:
!pip install transformers[sentencepiece]
!unzip /content/drive/MyDrive/Packages/bltk.zip -d "/content/"

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Dataset/text_summarization.csv")
print(df.loc[0][2])
text = df.loc[0][2]

ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙামাটি জেলার সব আদালতের মামলার সংক্রান্ত তথ্য এখন থেকে মোবাইল অ্যাপসে পাওয়া যাবে।আজ শনিবার ঢাকা মহানগর দায়রা জজ আদালতের জগন্নাথ-সোহেল স্মৃতি মিলনায়তনে এই অ্যাপসের আনুষ্ঠানিক উদ্বোধন করেন প্রধান বিচারপতি সুরেন্দ্র কুমার সিনহা।এই অ্যাপস তৈরির সঙ্গে যুক্ত ইউএনডিপির আইসিটি ম্যানেজার রিয়াজুল আল মাসুম প্রথম আলোকে বলেন, অ্যান্ড্রয়েড মোবাইলের গুগল প্লে স্টোর থেকে ‘বিডি কোর্ট’ নামের অ্যাপসটি ডাউনলোড করা যাবে। তিনি জানান, এই অ্যাপসে আদালতের প্রতিদিনের মামলার কার্যতালিকা, মামলার বিস্তারিত বিবরণ, মামলার ফলাফলসহ অন্যান্য প্রয়োজনীয় তথ্য পাওয়া যাবে। দেশের সব আদালতের মামলা সংক্রান্ত তথ্য এই অ্যাপসের আওতায় আনার কাজ চলছে।মোবাইল অ্যাপসে গিয়ে দেখা গেছে, ‘কজ লিস্ট’ (কার্যতালিকা) ও ‘কেস সার্স’ (মামলা অনুসন্ধান) নামে দুটি বাটন রয়েছে। ‘কজ লিস্ট’ বাটনে ক্লিক করে আদালতের প্রতিদিনকার মামলার তালিকা পাওয়া যাচ্ছে। আর ‘কেস সার্স’ বাটনে ক্লিক করে মামলার পুরো বিবরণ জানা যাচ্ছে।উদ্বোধনী অনুষ্ঠানে ঢাকার জেলা ও দায়রা জজ এস এম কুদ্দুস জামান বলেন, এই অ্যাপসের মাধ্যমে সাধারণ মানু

In [ ]:
from typing import List
from bltk.langtools.banglachars import punctuations
from bltk.langtools import Tokenizer as BLTKTokenizer
from tokenizers import Tokenizer, Regex, NormalizedString, PreTokenizedString
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import PreTokenizer
from tokenizers.normalizers import Normalizer
from tokenizers.decoders import Decoder

bltkTokenizer = BLTKTokenizer()

def custom_tokenizer(text):
    string_tokenized = []
    start = 0
    sent_tokens=list(map(lambda x:x.strip(),df.loc[0][2].split("।")))[:-1]

    for sent in sent_tokens:
        words = bltkTokenizer.word_tokenizer(sent)
        for word in words:
            if word not in punctuations:
                if word[0] in punctuations:
                    word = word[1:]
                if word[-1] in punctuations:
                    word = word[:-1]
                end = start+len(word)
                string_tokenized.append((word, start, end))
                start = end+1
#     print(string_tokenized)
    return string_tokenized


class JiebaPreTokenizer:
    def jieba_split(self, i: int, normalized_string: NormalizedString) -> List[NormalizedString]:
        splits = []
        # we need to call `str(normalized_string)` because jieba expects a str,
        # not a NormalizedString
        for token, start, stop in custom_tokenizer(str(normalized_string)):
#             print(normalized_string)
            splits.append(normalized_string[start:stop])

        return splits
        # We can also easily do it in one line:
        # return [normalized_string[w[1] : w[2]] for w in jieba.tokenize(str(normalized_string))]

    def odd_number_split(
        self, i: int, normalized_string: NormalizedString
    ) -> List[NormalizedString]:
        # Just an odd example...
        splits = []
        last = 0
        for (i, char) in enumerate(str(normalized_string)):
            if char.isnumeric() and int(char) % 2 == 1:
                splits.append(normalized_string[last:i])
                last = i
        # Don't forget the last one
        splits.append(normalized_string[last:])
        return splits

    def pre_tokenize(self, pretok: PreTokenizedString):
        # Let's call split on the PreTokenizedString to split using `self.jieba_split`
        pretok.split(self.jieba_split)
        # Here we can call `pretok.split` multiple times if we want to apply
        # different algorithm, but we generally just need to call it once.
        pretok.split(self.odd_number_split)


class CustomDecoder:
    def decode(self, tokens: List[str]) -> str:
        return "".join(tokens)


class CustomNormalizer:
    def normalize(self, normalized: NormalizedString):
        # Most of these can be replaced by a `Sequence` combining some provided Normalizer,
        # (ie Sequence([ NFKC(), Replace(Regex("\s+"), " "), Lowercase() ])
        # and it should be the prefered way. That being said, here is an example of the kind
        # of things that can be done here:
        normalized.nfkc()
        normalized.filter(lambda char: not char.isnumeric())
        normalized.replace(Regex("\s+"), " ")
        normalized.lowercase()


# This section shows how to attach these custom components to the Tokenizer
tok = Tokenizer(BPE())
tok.normalizer = Normalizer.custom(CustomNormalizer())
tok.pre_tokenizer = PreTokenizer.custom(JiebaPreTokenizer())
tok.decoder = Decoder.custom(CustomDecoder())

# input = "永和服装饰品有限公司"
print("PreTokenize:", text)
print(tok.pre_tokenizer.pre_tokenize_str(text))

PreTokenize: ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙামাটি জেলার সব আদালতের মামলার সংক্রান্ত তথ্য এখন থেকে মোবাইল অ্যাপসে পাওয়া যাবে।আজ শনিবার ঢাকা মহানগর দায়রা জজ আদালতের জগন্নাথ-সোহেল স্মৃতি মিলনায়তনে এই অ্যাপসের আনুষ্ঠানিক উদ্বোধন করেন প্রধান বিচারপতি সুরেন্দ্র কুমার সিনহা।এই অ্যাপস তৈরির সঙ্গে যুক্ত ইউএনডিপির আইসিটি ম্যানেজার রিয়াজুল আল মাসুম প্রথম আলোকে বলেন, অ্যান্ড্রয়েড মোবাইলের গুগল প্লে স্টোর থেকে ‘বিডি কোর্ট’ নামের অ্যাপসটি ডাউনলোড করা যাবে। তিনি জানান, এই অ্যাপসে আদালতের প্রতিদিনের মামলার কার্যতালিকা, মামলার বিস্তারিত বিবরণ, মামলার ফলাফলসহ অন্যান্য প্রয়োজনীয় তথ্য পাওয়া যাবে। দেশের সব আদালতের মামলা সংক্রান্ত তথ্য এই অ্যাপসের আওতায় আনার কাজ চলছে।মোবাইল অ্যাপসে গিয়ে দেখা গেছে, ‘কজ লিস্ট’ (কার্যতালিকা) ও ‘কেস সার্স’ (মামলা অনুসন্ধান) নামে দুটি বাটন রয়েছে। ‘কজ লিস্ট’ বাটনে ক্লিক করে আদালতের প্রতিদিনকার মামলার তালিকা পাওয়া যাচ্ছে। আর ‘কেস সার্স’ বাটনে ক্লিক করে মামলার পুরো বিবরণ জানা যাচ্ছে।উদ্বোধনী অনুষ্ঠানে ঢাকার জেলা ও দায়রা জজ এস এম কুদ্দুস জামান বলেন, এই অ্যাপসের মাধ্যম

In [ ]:
# from tokenizers import (
#     decoders,
#     models,
#     normalizers,
#     pre_tokenizers,
#     processors,
#     trainers,
#     Tokenizer,
# )


In [ ]:
# from tokenizers.trainers import BpeTrainer
# def create_trainer():
#     return BpeTrainer(vocab_size=15000, show_progress=True,
#                     min_frequency=0, special_tokens=[
#                     "<s>",
#                     "<pad>",
#                     "</s>",
#                     "<unk>",
#                     "<mask>",
#                     ]
#           )
# trainer = create_trainer()
# trainer = create_trainer()
# tok.train(trainer=trainer, files=["/content/d.txt"])

In [ ]:
# tok.model.save('.')

['./vocab.json', './merges.txt']

In [ ]:
# output = tok.encode(text)
# print(output.tokens)

['ঢাকা', 'মহানগর', 'ও', 'ঢাকা', 'জেলা', 'আদালত', ' কিশোরগঞ্', ' ', ' রাঙামাট', ' জেলা', ' স', ' আদালতে', ' মামলা', ' সংক্রান্', ' তথ্', ' এখ', ' থেক', ' মোবাই', ' অ্যাপস', ' পাওয', 'া যা', 'ে।', 'জ শনিব', 'র ঢা', 'া মহান', 'র দায', 'রা', 'জজ আদাল', 'ের জগন্নাথ-সো', 'েল স্ম', 'তি মিলনায', 'তন', ' এই অ্যা', 'সের আনুষ্ঠ', 'নিক উদ্', 'োধন ', 'রেন প্', 'ধান বিচা', 'পতি সুরেন', 'দ্র ক', 'মার স', 'নহ', '।এই অ্', 'াপস ত', 'রির স', '্গে য', 'ক্ত ইউএনড', 'পির আই', 'িটি ম্যান', 'জার রিয', 'াজ', 'ল আল ', 'াসুম ', '্রথম ', 'লোকে', 'বলেন, অ্যান্', '্রয়েড ম', 'বাইল', 'র গু', 'ল প্ল', ' স্ট', 'র থেক', ' ‘বিডি', 'কোর্ট', ' নামের অ', 'যাপসটি ', 'াউন', 'োড ক', 'া যা', 'ে। তি', 'ি ', 'ানান, এ', ' অ্যাপস', ' আদালতের প', 'রতিদিন', 'র মামলার কা', '্যতালি', 'া, মামলার', 'বিস্ত', 'রিত বি', 'রণ, মাম', 'ার ফলাফল', 'হ অন্যান্', ' প্র', '়োজনী', '় তথ', 'য পাও', '়া', 'যাবে। দ', 'শের স', ' আদালতের ', 'ামলা', 'সং', '্রান্ত ত', '্য এই', 'অ্যা', 'সের', 'আওতা', '় আনার', 'কাজ চলছ', '।মোব', 'ইল অ', 'যাপস', ' গি', '়ে 

In [ ]:
# from transformers import pipeline, BartTokenizer, BartForConditionalGeneration
# from transformers import BertTokenizerFast
# from transformers import PreTrainedTokenizerFast, LineByLineTextDataset
# from tokenizers.implementations import BaseTokenizer
# tokenizer = PreTrainedTokenizerFast(BaseTokenizer(tok))
# model_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# summerize_pipe_bart = pipeline("summarization", model='facebook/bart-base' , tokenizer=tokenizer)
# pipe_out_bart = summerize_pipe_bart(text, truncation=True)
# pipe_out_bart

ValueError: ignored

In [ ]:
def custom_tokenizer(text):
    string_tokenized = []
    start = 0
    sent_tokens=list(map(lambda x:x.strip(),df.loc[0][2].split("।")))[:-1]

    for sent in sent_tokens:
        words = bltkTokenizer.word_tokenizer(sent)
        for word in words:
            if word not in punctuations:
                if word[0] in punctuations:
                    word = word[1:]
                if word[-1] in punctuations:
                    word = word[:-1]
                end = start+len(word)
                string_tokenized.append(word)
                start = end+1
    # print(string_tokenized)
    return string_tokenized
# custom_tokenizer(text)

['ঢাকা', 'মহানগর', 'ও', 'ঢাকা', 'জেলা', 'আদালত', 'কিশোরগঞ্জ', 'ও', 'রাঙামাটি', 'জেলার', 'সব', 'আদালতের', 'মামলার', 'সংক্রান্ত', 'তথ্য', 'এখন', 'থেকে', 'মোবাইল', 'অ্যাপসে', 'পাওয়া', 'যাবে', 'আজ', 'শনিবার', 'ঢাকা', 'মহানগর', 'দায়রা', 'জজ', 'আদালতের', 'জগন্নাথ-সোহেল', 'স্মৃতি', 'মিলনায়তনে', 'এই', 'অ্যাপসের', 'আনুষ্ঠানিক', 'উদ্বোধন', 'করেন', 'প্রধান', 'বিচারপতি', 'সুরেন্দ্র', 'কুমার', 'সিনহা', 'এই', 'অ্যাপস', 'তৈরির', 'সঙ্গে', 'যুক্ত', 'ইউএনডিপির', 'আইসিটি', 'ম্যানেজার', 'রিয়াজুল', 'আল', 'মাসুম', 'প্রথম', 'আলোকে', 'বলেন', 'অ্যান্ড্রয়েড', 'মোবাইলের', 'গুগল', 'প্লে', 'স্টোর', 'থেকে', '‘বিডি', 'কোর্ট’', 'নামের', 'অ্যাপসটি', 'ডাউনলোড', 'করা', 'যাবে', 'তিনি', 'জানান', 'এই', 'অ্যাপসে', 'আদালতের', 'প্রতিদিনের', 'মামলার', 'কার্যতালিকা', 'মামলার', 'বিস্তারিত', 'বিবরণ', 'মামলার', 'ফলাফলসহ', 'অন্যান্য', 'প্রয়োজনীয়', 'তথ্য', 'পাওয়া', 'যাবে', 'দেশের', 'সব', 'আদালতের', 'মামলা', 'সংক্রান্ত', 'তথ্য', 'এই', 'অ্যাপসের', 'আওতায়', 'আনার', 'কাজ', 'চলছে', 'মোবাইল', 'অ্যাপসে', 'গিয়ে', 'দেখা', 'গেছে', '‘কজ', 'লিস

['ঢাকা',
 'মহানগর',
 'ও',
 'ঢাকা',
 'জেলা',
 'আদালত',
 'কিশোরগঞ্জ',
 'ও',
 'রাঙামাটি',
 'জেলার',
 'সব',
 'আদালতের',
 'মামলার',
 'সংক্রান্ত',
 'তথ্য',
 'এখন',
 'থেকে',
 'মোবাইল',
 'অ্যাপসে',
 'পাওয়া',
 'যাবে',
 'আজ',
 'শনিবার',
 'ঢাকা',
 'মহানগর',
 'দায়রা',
 'জজ',
 'আদালতের',
 'জগন্নাথ-সোহেল',
 'স্মৃতি',
 'মিলনায়তনে',
 'এই',
 'অ্যাপসের',
 'আনুষ্ঠানিক',
 'উদ্বোধন',
 'করেন',
 'প্রধান',
 'বিচারপতি',
 'সুরেন্দ্র',
 'কুমার',
 'সিনহা',
 'এই',
 'অ্যাপস',
 'তৈরির',
 'সঙ্গে',
 'যুক্ত',
 'ইউএনডিপির',
 'আইসিটি',
 'ম্যানেজার',
 'রিয়াজুল',
 'আল',
 'মাসুম',
 'প্রথম',
 'আলোকে',
 'বলেন',
 'অ্যান্ড্রয়েড',
 'মোবাইলের',
 'গুগল',
 'প্লে',
 'স্টোর',
 'থেকে',
 '‘বিডি',
 'কোর্ট’',
 'নামের',
 'অ্যাপসটি',
 'ডাউনলোড',
 'করা',
 'যাবে',
 'তিনি',
 'জানান',
 'এই',
 'অ্যাপসে',
 'আদালতের',
 'প্রতিদিনের',
 'মামলার',
 'কার্যতালিকা',
 'মামলার',
 'বিস্তারিত',
 'বিবরণ',
 'মামলার',
 'ফলাফলসহ',
 'অন্যান্য',
 'প্রয়োজনীয়',
 'তথ্য',
 'পাওয়া',
 'যাবে',
 'দেশের',
 'সব',
 'আদালতের',
 'মামলা',
 'সংক্রান্ত',
 'তথ্য',
 'এই',
 'অ্যাপস

In [ ]:
words_tokenized = custom_tokenizer(text)

['ঢাকা', 'মহানগর', 'ও', 'ঢাকা', 'জেলা', 'আদালত', 'কিশোরগঞ্জ', 'ও', 'রাঙামাটি', 'জেলার', 'সব', 'আদালতের', 'মামলার', 'সংক্রান্ত', 'তথ্য', 'এখন', 'থেকে', 'মোবাইল', 'অ্যাপসে', 'পাওয়া', 'যাবে', 'আজ', 'শনিবার', 'ঢাকা', 'মহানগর', 'দায়রা', 'জজ', 'আদালতের', 'জগন্নাথ-সোহেল', 'স্মৃতি', 'মিলনায়তনে', 'এই', 'অ্যাপসের', 'আনুষ্ঠানিক', 'উদ্বোধন', 'করেন', 'প্রধান', 'বিচারপতি', 'সুরেন্দ্র', 'কুমার', 'সিনহা', 'এই', 'অ্যাপস', 'তৈরির', 'সঙ্গে', 'যুক্ত', 'ইউএনডিপির', 'আইসিটি', 'ম্যানেজার', 'রিয়াজুল', 'আল', 'মাসুম', 'প্রথম', 'আলোকে', 'বলেন', 'অ্যান্ড্রয়েড', 'মোবাইলের', 'গুগল', 'প্লে', 'স্টোর', 'থেকে', '‘বিডি', 'কোর্ট’', 'নামের', 'অ্যাপসটি', 'ডাউনলোড', 'করা', 'যাবে', 'তিনি', 'জানান', 'এই', 'অ্যাপসে', 'আদালতের', 'প্রতিদিনের', 'মামলার', 'কার্যতালিকা', 'মামলার', 'বিস্তারিত', 'বিবরণ', 'মামলার', 'ফলাফলসহ', 'অন্যান্য', 'প্রয়োজনীয়', 'তথ্য', 'পাওয়া', 'যাবে', 'দেশের', 'সব', 'আদালতের', 'মামলা', 'সংক্রান্ত', 'তথ্য', 'এই', 'অ্যাপসের', 'আওতায়', 'আনার', 'কাজ', 'চলছে', 'মোবাইল', 'অ্যাপসে', 'গিয়ে', 'দেখা', 'গেছে', '‘কজ', 'লিস

In [ ]:
from bltk.langtools.banglachars import punctuations
word_freq={}
for word in words_tokenized:
    if word not in punctuations:
        if word_freq.get(word) is None:
            word_freq[word] = 1
        else:
            word_freq[word] += 1
print(word_freq)


{'ঢাকা': 3, 'মহানগর': 2, 'ও': 4, 'জেলা': 2, 'আদালত': 1, 'কিশোরগঞ্জ': 1, 'রাঙামাটি': 1, 'জেলার': 1, 'সব': 2, 'আদালতের': 5, 'মামলার': 7, 'সংক্রান্ত': 2, 'তথ্য': 4, 'এখন': 1, 'থেকে': 2, 'মোবাইল': 2, 'অ্যাপসে': 3, 'পাওয়া': 3, 'যাবে': 3, 'আজ': 1, 'শনিবার': 1, 'দায়রা': 2, 'জজ': 2, 'জগন্নাথ-সোহেল': 1, 'স্মৃতি': 1, 'মিলনায়তনে': 1, 'এই': 5, 'অ্যাপসের': 3, 'আনুষ্ঠানিক': 1, 'উদ্বোধন': 1, 'করেন': 1, 'প্রধান': 1, 'বিচারপতি': 1, 'সুরেন্দ্র': 1, 'কুমার': 1, 'সিনহা': 1, 'অ্যাপস': 1, 'তৈরির': 1, 'সঙ্গে': 1, 'যুক্ত': 1, 'ইউএনডিপির': 1, 'আইসিটি': 1, 'ম্যানেজার': 1, 'রিয়াজুল': 1, 'আল': 1, 'মাসুম': 1, 'প্রথম': 1, 'আলোকে': 1, 'বলেন': 2, 'অ্যান্ড্রয়েড': 1, 'মোবাইলের': 1, 'গুগল': 1, 'প্লে': 1, 'স্টোর': 1, '‘বিডি': 1, 'কোর্ট’': 1, 'নামের': 1, 'অ্যাপসটি': 1, 'ডাউনলোড': 1, 'করা': 1, 'তিনি': 1, 'জানান': 1, 'প্রতিদিনের': 1, 'কার্যতালিকা': 2, 'বিস্তারিত': 1, 'বিবরণ': 2, 'ফলাফলসহ': 1, 'অন্যান্য': 1, 'প্রয়োজনীয়': 1, 'দেশের': 1, 'মামলা': 2, 'আওতায়': 1, 'আনার': 1, 'কাজ': 1, 'চলছে': 1, 'গিয়ে': 1, 'দেখা': 1, 'গেছে': 1, '

In [ ]:
x=(word_freq.values())
a=list(x)
a.sort()
max_freq=a[-1]
max_freq

7

In [ ]:
for word in word_freq.keys():
    word_freq[word]=word_freq[word]/max_freq
print(word_freq)

{'ঢাকা': 0.42857142857142855, 'মহানগর': 0.2857142857142857, 'ও': 0.5714285714285714, 'জেলা': 0.2857142857142857, 'আদালত': 0.14285714285714285, 'কিশোরগঞ্জ': 0.14285714285714285, 'রাঙামাটি': 0.14285714285714285, 'জেলার': 0.14285714285714285, 'সব': 0.2857142857142857, 'আদালতের': 0.7142857142857143, 'মামলার': 1.0, 'সংক্রান্ত': 0.2857142857142857, 'তথ্য': 0.5714285714285714, 'এখন': 0.14285714285714285, 'থেকে': 0.2857142857142857, 'মোবাইল': 0.2857142857142857, 'অ্যাপসে': 0.42857142857142855, 'পাওয়া': 0.42857142857142855, 'যাবে': 0.42857142857142855, 'আজ': 0.14285714285714285, 'শনিবার': 0.14285714285714285, 'দায়রা': 0.2857142857142857, 'জজ': 0.2857142857142857, 'জগন্নাথ-সোহেল': 0.14285714285714285, 'স্মৃতি': 0.14285714285714285, 'মিলনায়তনে': 0.14285714285714285, 'এই': 0.7142857142857143, 'অ্যাপসের': 0.42857142857142855, 'আনুষ্ঠানিক': 0.14285714285714285, 'উদ্বোধন': 0.14285714285714285, 'করেন': 0.14285714285714285, 'প্রধান': 0.14285714285714285, 'বিচারপতি': 0.14285714285714285, 'সুরেন্দ্র': 0.

In [ ]:
sent_score={}

print(sent_tokens)
for sent in sent_tokens:
    if sent_score.get(sent) is None:
        sent_score[sent] = 0.0
    for word in process_words(tokenizer.word_tokenizer(sent)):
        sent_score[sent] += word_freq[word]
print(sent_score)

['ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙামাটি জেলার সব আদালতের মামলার সংক্রান্ত তথ্য এখন থেকে মোবাইল অ্যাপসে পাওয়া যাবে', 'আজ শনিবার ঢাকা মহানগর দায়রা জজ আদালতের জগন্নাথ-সোহেল স্মৃতি মিলনায়তনে এই অ্যাপসের আনুষ্ঠানিক উদ্বোধন করেন প্রধান বিচারপতি সুরেন্দ্র কুমার সিনহা', 'এই অ্যাপস তৈরির সঙ্গে যুক্ত ইউএনডিপির আইসিটি ম্যানেজার রিয়াজুল আল মাসুম প্রথম আলোকে বলেন, অ্যান্ড্রয়েড মোবাইলের গুগল প্লে স্টোর থেকে ‘বিডি কোর্ট’ নামের অ্যাপসটি ডাউনলোড করা যাবে', 'তিনি জানান, এই অ্যাপসে আদালতের প্রতিদিনের মামলার কার্যতালিকা, মামলার বিস্তারিত বিবরণ, মামলার ফলাফলসহ অন্যান্য প্রয়োজনীয় তথ্য পাওয়া যাবে', 'দেশের সব আদালতের মামলা সংক্রান্ত তথ্য এই অ্যাপসের আওতায় আনার কাজ চলছে', 'মোবাইল অ্যাপসে গিয়ে দেখা গেছে, ‘কজ লিস্ট’ (কার্যতালিকা) ও ‘কেস সার্স’ (মামলা অনুসন্ধান) নামে দুটি বাটন রয়েছে', '‘কজ লিস্ট’ বাটনে ক্লিক করে আদালতের প্রতিদিনকার মামলার তালিকা পাওয়া যাচ্ছে', 'আর ‘কেস সার্স’ বাটনে ক্লিক করে মামলার পুরো বিবরণ জানা যাচ্ছে', 'উদ্বোধনী অনুষ্ঠানে ঢাকার জেলা ও দায়রা জজ এস এম কুদ্দুস জামান বলেন, এই অ্যাপস

In [ ]:
from heapq import nlargest
summary_size = int(len(sent_score) *0.3)

In [ ]:
summary=nlargest(n=summary_size,iterable=sent_score,key=sent_score.get)
print(summary)

['ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙামাটি জেলার সব আদালতের মামলার সংক্রান্ত তথ্য এখন থেকে মোবাইল অ্যাপসে পাওয়া যাবে', 'তিনি জানান, এই অ্যাপসে আদালতের প্রতিদিনের মামলার কার্যতালিকা, মামলার বিস্তারিত বিবরণ, মামলার ফলাফলসহ অন্যান্য প্রয়োজনীয় তথ্য পাওয়া যাবে', 'উদ্বোধনী অনুষ্ঠানে ঢাকার জেলা ও দায়রা জজ এস এম কুদ্দুস জামান বলেন, এই অ্যাপসের মাধ্যমে সাধারণ মানুষ মামলার সমস্ত তথ্য জানতে পারবে']


In [ ]:
final_summary = "।".join(summary)+"।"
print(final_summary)

ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙামাটি জেলার সব আদালতের মামলার সংক্রান্ত তথ্য এখন থেকে মোবাইল অ্যাপসে পাওয়া যাবে।তিনি জানান, এই অ্যাপসে আদালতের প্রতিদিনের মামলার কার্যতালিকা, মামলার বিস্তারিত বিবরণ, মামলার ফলাফলসহ অন্যান্য প্রয়োজনীয় তথ্য পাওয়া যাবে।উদ্বোধনী অনুষ্ঠানে ঢাকার জেলা ও দায়রা জজ এস এম কুদ্দুস জামান বলেন, এই অ্যাপসের মাধ্যমে সাধারণ মানুষ মামলার সমস্ত তথ্য জানতে পারবে।


In [ ]:
print(df.loc[0][1])

অ্যাপসে মিলবে ঢাকাসহ তিন জেলা আদালতের তথ্য


In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import rouge
from rouge import Rouge
def evaluate_rouge(reference_text, summary_text):
  rouge = Rouge()
  scores = rouge.get_scores(reference_text, summary_text)
  return scores[0]['rouge-1']['f']

In [ ]:
print(evaluate_rouge(final_summary,df.loc[0][1] ))

0.1538461515162722


# **Models Those Didnt Work Well**

In [ ]:
text

'ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙামাটি জেলার সব আদালতের মামলার সংক্রান্ত তথ্য এখন থেকে মোবাইল অ্যাপসে পাওয়া যাবে।আজ শনিবার ঢাকা মহানগর দায়রা জজ আদালতের জগন্নাথ-সোহেল স্মৃতি মিলনায়তনে এই অ্যাপসের আনুষ্ঠানিক উদ্বোধন করেন প্রধান বিচারপতি সুরেন্দ্র কুমার সিনহা।এই অ্যাপস তৈরির সঙ্গে যুক্ত ইউএনডিপির আইসিটি ম্যানেজার রিয়াজুল আল মাসুম প্রথম আলোকে বলেন, অ্যান্ড্রয়েড মোবাইলের গুগল প্লে স্টোর থেকে ‘বিডি কোর্ট’ নামের অ্যাপসটি ডাউনলোড করা যাবে। তিনি জানান, এই অ্যাপসে আদালতের প্রতিদিনের মামলার কার্যতালিকা, মামলার বিস্তারিত বিবরণ, মামলার ফলাফলসহ অন্যান্য প্রয়োজনীয় তথ্য পাওয়া যাবে। দেশের সব আদালতের মামলা সংক্রান্ত তথ্য এই অ্যাপসের আওতায় আনার কাজ চলছে।মোবাইল অ্যাপসে গিয়ে দেখা গেছে, ‘কজ লিস্ট’ (কার্যতালিকা) ও ‘কেস সার্স’ (মামলা অনুসন্ধান) নামে দুটি বাটন রয়েছে। ‘কজ লিস্ট’ বাটনে ক্লিক করে আদালতের প্রতিদিনকার মামলার তালিকা পাওয়া যাচ্ছে। আর ‘কেস সার্স’ বাটনে ক্লিক করে মামলার পুরো বিবরণ জানা যাচ্ছে।উদ্বোধনী অনুষ্ঠানে ঢাকার জেলা ও দায়রা জজ এস এম কুদ্দুস জামান বলেন, এই অ্যাপসের মাধ্যমে সাধারণ মান

In [ ]:
###Import PegasusModel and Tokenizer

from transformers import pipeline, PegasusTokenizer, PegasusForConditionalGeneration
# Named entity recognition pipeline, passing in a specific model and tokenizer
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-pubmed')
tokenizer =PegasusTokenizer.from_pretrained('google/pegasus-pubmed')

summerize_pipe = pipeline("summarization", model=model, tokenizer=tokenizer)
pipe_out = summerize_pipe(text, truncation=True)
pipe_out

[{'summary_text': ''}]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5", use_fast=False)

summerize_pipe = pipeline("summarization", model=model, tokenizer=tokenizer)
pipe_out = summerize_pipe(text, truncation=True)
pipe_out

[{'summary_text': 'সার্স বাটনের মাধ্যমে এই অ্যাপস ডাউনলোড করা যাবে। সার্স'}]

In [ ]:
from transformers import pipeline, BartTokenizer, BartForConditionalGeneration
# Named entity recognition pipeline, passing in a specific model and tokenizer
model_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
tokenizer_bart =BartTokenizer.from_pretrained('facebook/bart-base')

summerize_pipe_bart = pipeline("summarization", model='facebook/bart-base' , tokenizer=tokenizer_bart)
pipe_out_bart = summerize_pipe_bart(text, truncation=True)
pipe_out_bart

[{'summary_text': 'ঢাকা মহিনগর ও জেল ত্লত সংক কোবি আদাুত, রিূসী পৈস ব নৌল\u09c6 গ\u09caক।া. অ�'}]